In [1]:
file = open('alllines.txt','r')
text = file.read()
lines = text.lower().split('\n')

#Define words, vocabulary size and sequences of words as lines
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
words = text_to_word_sequence(text)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(words)
vocabulary_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(lines)

#Find subsequences 
subsequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
       subsequence = sequence[:i+1]
       subsequences.append(subsequence)

## Padding Your Sequences

You need to have equal sequences for training; for this, you will apply padding.
Write these lines of code to implement the padding needed:

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequence_length = max([len(sequence) for sequence in sequences])
sequences = pad_sequences(subsequences, maxlen=sequence_length, padding='pre')

## Encode the Target Labels

In [3]:
from keras.utils import to_categorical


x , y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocabulary_size)

MemoryError: Unable to allocate 67.6 GiB for an array with shape (709820, 25576) and data type float32

In [4]:
print(x.shape, y.shape)

(709820, 166) (709820,)


## Define an RNN

In [5]:
from keras.models import Sequential
model = Sequential()

In [6]:
from keras.layers import Embedding, LSTM, Dropout, Dense

model.add(Embedding(input_dim = vocabulary_size,
                   output_dim = 100,
                   input_length = sequence_length - 1))
model.add(LSTM(units = 100))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = vocabulary_size, activation = "softmax"))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 166, 100)          2557600   
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 25576)             2583176   
                                                                 
Total params: 5,221,176
Trainable params: 5,221,176
Non-trainable params: 0
_________________________________________________________________


## Compile the Network

In [8]:
model.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics = ['accuracy'])

## Fit the Model

In [9]:
from livelossplot import PlotLossesKeras

model.fit(x, y,
         callbacks = [PlotLossesKeras()],
         epochs = 500)

Epoch 1/500


ValueError: in user code:

    File "C:\Users\angel\anaconda3\envs\pml\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\angel\anaconda3\envs\pml\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\angel\anaconda3\envs\pml\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\angel\anaconda3\envs\pml\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\angel\anaconda3\envs\pml\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "C:\Users\angel\anaconda3\envs\pml\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_v